### Setting

In [3]:
import os
import sys
from dotenv import load_dotenv

load_dotenv()
WORK_DIR = os.environ.get("WORK_DIR")
print(WORK_DIR)
# sys.path.append(WORK_DIR)

from notion_client import Client
NOTION_TOKEN = os.environ.get("NOTION_TOKEN")
NOTION_DB_ID = os.environ.get("NOTION_DB_ID")

/PaperSurvey


In [4]:
from pprint import pprint
notion_client = Client(auth=NOTION_TOKEN)

get_info_test = notion_client.databases.query(
    **{
        'database_id' :  NOTION_DB_ID # データベースID
       }
)
pprint(get_info_test)

{'has_more': False,
 'next_cursor': None,
 'object': 'list',
 'page_or_database': {},
 'request_id': 'f79eb041-2e37-4111-92af-816a3370c1ba',
 'results': [{'archived': False,
              'cover': None,
              'created_by': {'id': 'cfbe164d-c3b0-4830-9004-5cd83b3bea6c',
                             'object': 'user'},
              'created_time': '2024-12-17T08:02:00.000Z',
              'icon': None,
              'id': '15f986f2-ed0e-80e8-b351-ef0996de1b33',
              'in_trash': False,
              'last_edited_by': {'id': 'cfbe164d-c3b0-4830-9004-5cd83b3bea6c',
                                 'object': 'user'},
              'last_edited_time': '2024-12-17T08:02:00.000Z',
              'object': 'page',
              'parent': {'database_id': '15f986f2-ed0e-80ab-9b9e-e20186f3e5ed',
                         'type': 'database_id'},
              'properties': {'title': {'id': 'title',
                                       'title': [{'annotations': {'bold': False,
      

In [9]:
notion_client.pages.create(
        **{
            'parent': {'database_id': NOTION_DB_ID},
            'properties': {
                'title': {
                    'title': [
                        {
                            'text': {
                                'content': 'test3'
                            }
                        }
                    ]
                },
            },
            'children': [
                {
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [
                            {
                                'type': 'text',
                                'text': {
                                    'content': "# test3"
                                }
                            }
                        ]
                    }
                }
            ]
        }
    )


{'object': 'page',
 'id': '15f986f2-ed0e-81cb-9348-c3615fd406b4',
 'created_time': '2024-12-17T08:24:00.000Z',
 'last_edited_time': '2024-12-17T08:24:00.000Z',
 'created_by': {'object': 'user',
  'id': 'cc1f5e8b-355f-422e-9bf5-b5a48742e6c9'},
 'last_edited_by': {'object': 'user',
  'id': 'cc1f5e8b-355f-422e-9bf5-b5a48742e6c9'},
 'cover': None,
 'icon': None,
 'parent': {'type': 'database_id',
  'database_id': '15f986f2-ed0e-80ab-9b9e-e20186f3e5ed'},
 'archived': False,
 'in_trash': False,
 'properties': {'title': {'id': 'title',
   'type': 'title',
   'title': [{'type': 'text',
     'text': {'content': 'test3', 'link': None},
     'annotations': {'bold': False,
      'italic': False,
      'strikethrough': False,
      'underline': False,
      'code': False,
      'color': 'default'},
     'plain_text': 'test3',
     'href': None}]}},
 'url': 'https://www.notion.so/test3-15f986f2ed0e81cb9348c3615fd406b4',
 'public_url': None,
 'request_id': '251a9ddc-bbdf-49a7-a5eb-e6c5483aecc7'}

In [8]:
from notion_client import Client

def create_markdown_page(notion, database_id, title, markdown_content):
    """
    マークダウンコンテンツを含むNotionページを作成する
    
    Parameters:
    - notion: Notion Client インスタンス
    - database_id: データベースID
    - title: ページタイトル
    - markdown_content: マークダウンコンテンツ
    """
    
    # マークダウンをNotionブロックに変換する関数
    def convert_heading_to_blocks(text, level):
        return {
            'object': 'block',
            'type': f'heading_{level}',
            f'heading_{level}': {
                'rich_text': [{'type': 'text', 'text': {'content': text}}]
            }
        }
    
    # コンテンツをブロックに変換
    blocks = []
    current_text = []
    
    for line in markdown_content.split('\n'):
        # 見出し1の処理 (#)
        if line.startswith('# '):
            if current_text:
                blocks.append({
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
                    }
                })
                current_text = []
            blocks.append(convert_heading_to_blocks(line[2:].strip(), 1))
            
        # 見出し2の処理 (##)
        elif line.startswith('## '):
            if current_text:
                blocks.append({
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
                    }
                })
                current_text = []
            blocks.append(convert_heading_to_blocks(line[3:].strip(), 2))
            
        # 空行の処理
        elif line.strip() == '':
            if current_text:
                blocks.append({
                    'object': 'block',
                    'type': 'paragraph',
                    'paragraph': {
                        'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
                    }
                })
                current_text = []
            
        # 通常のテキスト処理
        else:
            current_text.append(line)
    
    # 残りのテキストを処理
    if current_text:
        blocks.append({
            'object': 'block',
            'type': 'paragraph',
            'paragraph': {
                'rich_text': [{'type': 'text', 'text': {'content': '\n'.join(current_text)}}]
            }
        })

    # ページの作成
    return notion.pages.create(
        **{
            'parent': {'database_id': database_id},
            'properties': {
                'title': {
                    'title': [
                        {
                            'text': {
                                'content': title
                            }
                        }
                    ]
                }
            },
            'children': blocks
        }
    )

# 使用例
# notion = Client(auth="your-notion-api-key")
# database_id = "your-database-id"
title = "Learning Community Embedding with Community Detection and Node Embedding on Graphs"
markdown_content = """# abstract
この論文では、グラフ埋め込みの重要でありながらほとんど未開拓の設定である、個々のノードではなくコミュニティを埋め込むことを研究しています。コミュニティ埋め込みは、グラフの視覚化などのコミュニティレベルのアプリケーションに有用であるだけでなく、コミュニティ検出やノード分類にも有益です。この埋め込みを学習するために、コミュニティ埋め込み、コミュニティ検出、ノード埋め込みの間に閉ループが存在するという洞察に基づいています。提案するフレームワークは、これら3つのタスクを同時に解決します。

# 解決する課題
## 既存研究の流れ（関連研究）
従来のグラフ埋め込みは、個々のノードに焦点を当て、グラフ上で「近い」ノードが低次元空間で類似したベクトル表現を持つことを目指していました。しかし、コミュニティ全体を埋め込むことはあまり研究されていませんでした。

## この研究が解決する課題・どう解決するのか
解決する課題1: コミュニティ埋め込みの有用性を示す
　→ コミュニティ埋め込みがグラフ視覚化やコミュニティ検出、ノード分類に有益であることを示す。
解決する課題2: コミュニティ検出とノード埋め込みの相互作用を最適化
　→ コミュニティ埋め込みを用いてノード埋め込みを最適化し、コミュニティ検出を改善する。
解決する課題3: 効率的なコミュニティ埋め込みフレームワークの提案
　→ コミュニティ検出、コミュニティ埋め込み、ノード埋め込みを同時に解決するフレームワークを提案。

# 提案手法
## 提案手法の直感的な説明
コミュニティ埋め込みを多変量ガウス分布として定義し、ノード埋め込み結果からコミュニティ検出を強化するためにガウス混合モデルを使用します。

## 提案手法詳細
提案手法「ComE」は、コミュニティ検出、コミュニティ埋め込み、ノード埋め込みを同時に解決するフレームワークです。コミュニティ埋め込みを多変量ガウス分布として定義し、ノード埋め込み結果からコミュニティ検出を強化します。計算の複雑さはグラフサイズに対して線形です。

# 実験
## 実験設定
5つの公開グラフデータセットを使用し、グラフ視覚化、コミュニティ検出、ノード分類の3つの評価タスクを設計しました。

## 実験結果
ComEは、グラフ視覚化、コミュニティ検出、ノード分類の各タスクで、最先端のベースラインを上回る結果を示しました。

# 考察
## なぜこの手法が優れているのか
コミュニティ検出、コミュニティ埋め込み、ノード埋め込みを同時に最適化することで、各タスクが相互に強化されるため。

## この手法が既存のものより優れている点・劣っている点
優れている点: コミュニティ検出とノード分類の精度が向上し、グラフ視覚化が改善される。
劣っている点: 特に記載なし。

# 今後の発展
論文には書かれていませんでした。
"""

page = create_markdown_page(notion_client, NOTION_DB_ID, title, markdown_content)